In [ ]:
import os
os.environ["SPARK_HOME"] ='C:/Users/kan_2/spark-2.4.8-bin-hadoop2.7'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.8 pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars C:/Users/kan_2/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [ ]:
import findspark
findspark.init()

In [ ]:
import pandas as pd

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json
from confluent_kafka import Producer
import uuid

In [ ]:
sc = pyspark.SparkContext()
spark = SparkSession(sc)
ssc = StreamingContext(sc,6) # Show results every 2 seconds

In [ ]:
kafka_topic_name = "jjmm_binance"
kafka_topic_name2 = "jjmm_bitkub"
kafka_topic_name3 = "jjmm_exchange"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [ ]:
def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {}'.format(msg.value().decode('utf-8')))

 #file1.readlines()

In [ ]:
def produce_kafka(msg):
    ID=uuid.uuid1()
    p = Producer({'bootstrap.servers': 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'})
    ID=uuid.uuid1()
    p.poll(0)
    if type(msg)!=str:
        msg=str(msg)
    sendMsg = msg.encode().decode('utf-8').strip('\n')
    p.produce('jjmm-stream', key="id{}".format(ID.hex), value=sendMsg , callback=delivery_report)
    p.flush()

In [ ]:
def readMyStream(rdd):
  if not rdd.isEmpty():
    df = spark.read.json(rdd)
    print(df.dtypes)
    print('Started the Process')
    print('Selection of Columns')
    df = df.select('firstname','lastname')
    df.show()

In [ ]:
def readMyStream2(rdd):
  if not rdd.isEmpty():
    df = spark.read.json(rdd)
    #df.write.mode('Overwrite').json("/Users/kan_2/spark_output/zipcodes")
    #df=resultDF.to_json()
    #df.write.mode('append').json("C:/6005/test.json")
#    df2 = spark.read.json(rdd2)

    # The inferred schema can be visualized using the printSchema() method
    df.printSchema()
#    df2.printSchema()
    # Creates a temporary view using the DataFrame
    df.createOrReplaceTempView("T1")

    # SQL statements can be run by using the sql methods provided by spark
    resultDF = spark.sql("SELECT * FROM T1")
    resultDF = resultDF.toPandas()
    result_csv = resultDF['timestamp'].to_list()
    a=result_csv[0].replace("/","-")
    b=a.replace(":","-")
    resultDF.to_csv("C:/Users/kan_2/testoutput"+b+".csv",index=False)
    c=resultDF.to_dict('records')[0]
    produce_kafka(c)
    
    #final_df.append(resultDF)
    #resultDF.show()
   


In [ ]:
def readMyStream3(rdd1,rdd2):
  if not rdd1.isEmpty() and not rdd2.isEmpty():
    df1 = spark.read.json(rdd1)
    #df1.printSchema()
    df2 = spark.read.json(rdd2)
    #df2.printSchema()
    #df3 = spark.read.json(rdd3)
    #df3.printSchema()
    
    df1.createOrReplaceTempView("binance")
    df2.createOrReplaceTempView("exchange")
    #df3.createOrReplaceTempView("exchange")

    df1 = df1.sort(desc('timestamp'))
    df2 = df2.sort(desc('timestamp'))
    #df3 = df3.sort(desc('timestamp'))
    
    #df1.show()
    #df2.show()
    A = spark.sql("SELECT timestamp,priceChange,openPrice,highPrice,lowPrice,lastPrice FROM binance limit 1")
    B = spark.sql("SELECT rate,symbol FROM exchange limit 1")
    A_pd = A.toPandas()
    B_pd = B.toPandas()
    out = pd.concat([A_pd,B_pd],axis=1)
    out['last_price(THB)'] = out['lastPrice']*out['rate']
    out['priceChange(THB)'] = out['priceChange']*out['rate']
    out['openPrice(THB)'] = out['openPrice']*out['rate']
    out['highPrice(THB)'] = out['highPrice']*out['rate']
    out['lowPrice(THB)'] = out['lowPrice']*out['rate']
    #out.to_csv("C:/Users/kan_2/testoutput2.csv",index=False)
    c=out.to_dict('records')[0]
    produce_kafka(c)
    #df3.show()
    
    #resultDF = df1.join(df2, df1.id == df2.id, 'outer').select(df2.firstname, df1.hobby)#.sort(desc("name")).collect()
    #resultDF.show()
    # SQL statements can be run by using the sql methods provided by spark
    #resultDF = spark.sql("SELECT * FROM people")
    #resultDF.show()

In [ ]:
kvs1 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'
                        })
'''
kvs2 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name2], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'
                        })
'''
kvs3 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name3], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'
                        })
                       
#kvs1.pprint()
#kvs2.pprint()
#kvs3.pprint()
parsed1 = kvs1.map(lambda x: x[1])
#parsed2 = kvs2.map(lambda x: x[1])
parsed3 = kvs3.map(lambda x: x[1])
parsed1.transformWith(readMyStream3,parsed3).pprint()
#a = parsed1.foreachRDD( lambda rdd: readMyStream2(rdd) )
#b = parsed2.transform( lambda rdd: readMyStream2(rdd) ).pprint()



#wordCounts.transform(lambda rdd: rdd.join(spamInfoRDD).filter(...))

ssc.start()
ssc.awaitTermination()
ssc.stop()
sc.stop()